In [1]:
import sys, os
import pandas as pd
sys.path.append("../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as influx_Client
from Clust.clust.ingestion.mongo.mongo_client import MongoClient
db_client = influx_Client.InfluxClient(ins.CLUSTDataServer2)
mongo_client = MongoClient(ins.CLUSTMetaInfo2)

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

from Clust.clust.ML.common import ML_pipeline, tool
app_name= "energy" # "Hs2SwineFarmWithWeatherTime", "energy"

cpu is available.


## 1. Data Preparation

### 1-1. Parameter Setting

In [2]:
if app_name == "energy":
    model_purpose = 'regression'
    feature_X_list = ['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7',
       'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7',
       'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
    feature_y_list = ['value']
    split_mode ="windows_split"
    data_y_flag = True # 이미 만들어진 Y 데이터를 활용함
    
    
elif app_name == "Hs2SwineFarmWithWeatherTime":
    model_purpose = 'forecasting' 
    feature_X_list = ['Temperature', 'out_temp','sin_hour']
    feature_y_list = ['Temperature']
    split_mode = 'step_split'
    data_y_flag = False # Y데이터는 없음, X 에서 Y 데이터를 도출함
    
step = 'train'
bucket_name = 'integration' 
data_clean_level = 4

### 1-2. Data Ingestion
#### 1-2-1. Select data name 

In [3]:
dataset_name = model_purpose + '_' + app_name  
print(dataset_name)
data_name_X = dataset_name + '_cleanLevel' + str(data_clean_level)+'_'+step+'X'
data_name_y = dataset_name+'_cleanLevel' + str(data_clean_level)+'_'+ step+'y'
data_meta = mongo_client.get_document_by_json('integration', data_name_X, {'ms_name':data_name_X})[0]

regression_energy


In [4]:
data_meta

{'bucket_name': 'integration',
 'collection_name': 'regression_energy',
 'ms_name': 'regression_energy_cleanLevel4_trainX',
 'ingestion_type': 'multiple_ms_by_time',
 'ingestion_param': {'ms_list_info': [['life_indoor_environment',
    'humidityTrain_10min'],
   ['life_indoor_environment', 'temperatureTrain_10min'],
   ['weather_outdoor_environment', 'belgiumChieverseAirportTrain_10min']],
  'start_time': '2016-01-11',
  'end_time': '2016-04-15'},
 'processing_type': 'step_3',
 'process_param': {'refine_param': {'remove_duplication': {'flag': True},
   'static_frequency': {'flag': True, 'frequency': None}},
  'outlier_param': {'certain_error_to_NaN': {'flag': True},
   'uncertain_error_to_NaN': {'flag': True,
    'param': {'outlierDetectorConfig': [{'algorithm': 'IQR',
       'percentile': 99,
       'alg_parameter': {'weight': 100}}]}}},
  'imputation_param': {'flag': True,
   'imputation_method': [{'min': 0,
     'max': 2,
     'method': 'linear',
     'parameter': {}}],
   'total_no

#### 1-2-2. X-y Data Ingestion pipeline

In [5]:
# Data Ingestion
ingestion_method = 'ms_all'
ingestion_param_X = {
    "bucket_name" : bucket_name,
    'ms_name' : data_name_X,
    'feature_list' : feature_X_list                              
}
ingestion_param_y = {
    "bucket_name" : bucket_name,
    'ms_name' : data_name_y,
    'feature_list' : feature_y_list                              
}

In [6]:
data_X, data_y = ML_pipeline.Xy_data_preparation(ingestion_param_X, data_y_flag, ingestion_param_y, ingestion_method, db_client)

{'bucket_name': 'integration', 'ms_name': 'regression_energy_cleanLevel4_trainX', 'feature_list': ['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']}


### 1-2-3. Data scaling

### Scaler save to server

In [7]:
def dirs_setting(sftp, dirs_path):
    dirs = dirs_path.split("/")
    path = ""
    
    for directory in dirs:
        path += "/" + directory
        try:
            sftp.mkdir(path)
        except OSError:
            pass

In [8]:
def scaler_file_save_to_server(scaler_path):
    import paramiko
    import pickle

    # Scaler 객체를 저장할 파일 경로
    remote_scaler_path = '/programdrive/Clust/file_save_test'
    scaler_server_path = remote_scaler_path + scaler_path[1:]
    scaler_local_path = os.getcwd() + scaler_path[1:]
    scaler_folder_path = scaler_server_path[:-10]

    # SSH 연결 설정
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect('10.252.107.59', username='jhlee', password='dlwlgns1')

    # Scaler 파일 전송
    sftp = ssh.open_sftp()
    sftp.chdir(remote_scaler_path)

    try:
        sftp.put(scaler_local_path, scaler_server_path)
    except IOError:
        dirs_setting(sftp, scaler_folder_path)
        sftp.chdir(scaler_folder_path)
        sftp.put(scaler_local_path, scaler_server_path)
    sftp.close()

    # SSH 연결 종료
    ssh.close()

In [9]:
scaler_param = {
        'scaler_flag':'scale', #scale_param,
        'scale_method' :'minmax',
        'scaler_path' :'./scaler/'
    }

In [10]:
def Xy_data_scaling_train(data_name_X, data_X, data_name_y, data_y, scaler_param):
    # X Data Scaling
    from Clust.clust.ML.tool import scaler
    scalerRootPath_X = os.path.join(scaler_param['scaler_path'], data_name_X)
    dataX_scaled, X_scalerFilePath = scaler.get_data_scaler(scaler_param['scaler_flag'], scalerRootPath_X, data_X, scaler_param['scale_method'])
    
    # y Data Scaling
    scalerRootPath_y = os.path.join(scaler_param['scaler_path'], data_name_y)
    datay_scaled, y_scalerFilePath = scaler.get_data_scaler(scaler_param['scaler_flag'], scalerRootPath_y, data_y, scaler_param['scale_method'])
    scaler_file_save_to_server(y_scalerFilePath)
    
    scaler_param['scaler_file_path'] = {
        "XScalerFile":{
            "fileName":"scaler.pkl",
            "filePath":X_scalerFilePath       
        },
        "yScalerFile":{
            "fileName":"scaler.pkl",
            "filePath":y_scalerFilePath
        }
    }

    return dataX_scaled, X_scalerFilePath, datay_scaled, y_scalerFilePath

In [11]:
dataX_scaled, X_scalerFilePath, datay_scaled, y_scalerFilePath= Xy_data_scaling_train(data_name_X, data_X, data_name_y, data_y, scaler_param)

['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
Make New scaler File
['value']
Make New scaler File


## 2. Cleaning and split
### 2.1 pipeline - clean low quality column

In [12]:
if split_mode =='windows_split':
    transform_parameter = {
        'split_mode' : 'window_split',
        'data_clean_option' : "false"
    }
else:
    transform_parameter = {
            'future_step': 2,
            'past_step': 12,
            'data_clean_option' : "false"
    }

NameError: name 'day_window_size' is not defined

In [ ]:

model_data_clean = True # Front End Parameter 
if model_data_clean:
    transform_parameter['nan_process_info'] = {'type':'num', 'ConsecutiveNanLimit':10, 'totalNaNLimit':100}
    max_nan_limit_ratio = 0.9
else:
    nan_process_info = {'type':'num', 'ConsecutiveNanLimit':10000, 'totalNaNLimit':100000}
    max_nan_limit_ratio = 0.5

nan_process_info = {'type':'num', 'ConsecutiveNanLimit':10000, 'totalNaNLimit':100000}

In [ ]:
dataX_scaled = ML_pipeline.clean_low_quality_column(dataX_scaled, transform_parameter)
feature_X_list= list(dataX_scaled.columns)

### 2.2 Train/Val Split pipeline

In [ ]:
split_ratio = 0.8

In [ ]:
# TODO 데이터 나뉘는 부분 추가로 작성된 것 지수님에게 물어봐야 함
# day_window_size = tool.get_default_day_window_size(dataX_scaled)
# train_x, val_x, train_y, val_y = ML_pipeline.split_data_by_mode(split_mode, split_ratio, dataX_scaled, datay_scaled, day_window_size)

train_x, val_x, train_y, val_y, transform_parameter = split_data_by_mode(dataX_scaled, datay_scaled, split_ratio, transform_parameter)

### 2.3 Data Transformation & Clean2 pipeline

In [ ]:
train_X_array, train_y_array = ML_pipeline.transform_data_by_split_mode(transform_parameter, train_x, train_y)
val_X_array, val_y_array = ML_pipeline.transform_data_by_split_mode(transform_parameter, val_x, val_y)

In [ ]:
print(train_X_array.shape)
print(train_y_array.shape)
print(val_X_array.shape)
print(val_y_array.shape)

### 2.4 Set Model and train parameter

In [ ]:
# RNN models (RNN, LSTM, GRU) parameters
seq_len, input_size = train_X_array.shape[1], train_X_array.shape[2]
model_method = 'GRU_rg' # Set model methods i.e., 'LSTM_rg', 'GRU_rg', 'CNN_1D_rg', 'LSTM_FCNs_rg', 'FC_rg' 

defalut_model_info ={"LSTM_rg":{"hidden_size":64,"num_layers":2,"output_dim":1,"dropout":0.1,"bidirectional":True},
                     "GRU_rg":{"hidden_size":64,"num_layers":2,"output_dim":1,"dropout":0.1,"bidirectional":True},
                     "CNN_1D_rg":{"output_channels":64,"kernel_size":3,"stride":1,"dropout":0.1,"padding":0},
                     "LSTM_FCNs_rg":{"num_layers":2,"lstm_dropout":0.4,"fc_dropout":0.1},
                     "FC_rg":{"dropout":2,"bias":0.4}}
default_train_param_info = {"lr":1e-4,"weight_decay":1e-6,"n_epochs":100,"batch_size":16}

model_info = defalut_model_info[model_method]

from Clust.clust.ML.common import parameter_setting
model_parameter = parameter_setting.set_model_parameter(model_method, model_info, seq_len, input_size)
train_parameter = parameter_setting.set_train_parameter(default_train_param_info)


In [ ]:
model_parameter

### 2.5 Set Model name and path pipeline

In [ ]:
model_name = None
model_file_path = None

# model name & path
if model_name is None:
    collection_index = dataset_name.find('_')
    app_name = dataset_name[collection_index:]
    model_name = tool.get_default_model_name(model_name, app_name, model_method, model_data_clean)
        
model_file_path = tool.get_default_model_path(model_name, data_name_X, model_method, train_parameter)

In [ ]:
model_name

In [ ]:
model_tags =["model_tag_example"]

## 3. Training

In [ ]:
model_meta = {
    'ingestion_param_X' : ingestion_param_X,
    'ingestion_param_y' : ingestion_param_y,
    'data_y_flag' : data_y_flag,
    'scaler_param':scaler_param,
    'transform_param':transform_param,
    'model_info':{
        'model_purpose' : model_purpose,
        'model_method' : model_method,
        'model_name' : model_name,
        'model_tags' : model_tags',
        'train_parameter' : train_parameter,
        'model_parameter' :model_parameter
    },
    'train_data_info' : data_meta
}

In [ ]:
ML_pipeline.CLUST_regresstion_train(train_X_array, train_y_array, val_X_array, val_y_array, model_meta['model_info'])

## 4. save metadata

In [ ]:
mongo_client.insert_document('model', 'JH_meta_Test', model_meta)